# Prepare Data

In [67]:
spark.sparkContext.hadoopConfiguration.set("fs.s3a.access.key", "<AWS_S3_ACCESS>")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.secret.key", "<AWS_S3_SECRET>")

In [46]:
import org.apache.spark.sql.functions._

In [25]:
val MIN_LAT = 30.0
val MAX_LAT = 50.0
val MIN_LON = -124.0
val MAX_LON = -74.0
val RES = 0.01

MIN_LAT = 30.0
MAX_LAT = 50.0
MIN_LON = -124.0
MAX_LON = -74.0
RES = 0.01


0.01

In [43]:
val llat = spark.sparkContext.parallelize(MIN_LAT until MAX_LAT by RES)
val llon = spark.sparkContext.parallelize(MIN_LON until MAX_LON by RES)

llat = ParallelCollectionRDD[35] at parallelize at <console>:33
llon = ParallelCollectionRDD[36] at parallelize at <console>:34


ParallelCollectionRDD[36] at parallelize at <console>:34

In [49]:
val df = llat.cartesian(llon).toDF("llat", "llon").withColumn("ulat", col("llat") + RES).withColumn("ulon", col("llon") + RES)

df = [llat: double, llon: double ... 2 more fields]


[llat: double, llon: double ... 2 more fields]

In [56]:
df.show(3)

+----+-------+-----+-------------------+
|llat|   llon| ulat|               ulon|
+----+-------+-----+-------------------+
|30.0| -124.0|30.01|            -123.99|
|30.0|-123.99|30.01|-123.97999999999999|
|30.0|-123.98|30.01|            -123.97|
+----+-------+-----+-------------------+
only showing top 3 rows



In [54]:
df.write.parquet("s3a://vz-raster-images/performance")

# Performance Test

In [1]:
import com.ibm.event.common.ConfigurationReader
import com.ibm.event.oltp.{EventContext,InsertResult}
import org.apache.spark.sql.ibm.event.EventSession
import com.ibm.event.catalog.{TableSchema,IndexSpecification,SortSpecification,ColumnOrder}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import scala.concurrent._
import scala.concurrent.duration._
import collection.JavaConverters._

import com.ibm.research.st.algorithms.hashing.eg.GeoHashEG
import com.ibm.research.st.datamodel.geometry.ellipsoidal.impl.BoundingBoxEG
import com.ibm.research.st.util.BitVector

### Event Context Configs

In [2]:
ConfigurationReader.setDeploymentID("db2eventstore-1583771517259")
ConfigurationReader.setEventUser("<EVENTSTORE_USER_NAME>")
ConfigurationReader.setEventPassword("<EVENTSTORE_PASSWORD>")
ConfigurationReader.setConnectionEndpoints("172.30.176.26:18729;172.30.176.26:1100")

val ctx = EventContext.getEventContext("EVENTDB")

ctx = com.ibm.event.oltp.EventContext@14c0ede


com.ibm.event.oltp.EventContext@14c0ede

In [3]:
val bitDepth = 29

bitDepth = 29


29

### Table Configs

In [4]:
val tabName = "performance"
val tabSchema = TableSchema(tabName, 
                            StructType(Array(
                                StructField("llat", DoubleType, nullable = false),
                                StructField("llon", DoubleType, nullable = false),
                                StructField("ulat", DoubleType, nullable = false),
                                StructField("ulon", DoubleType, nullable = false),
                                StructField("id", LongType, nullable = false),
                                StructField("geohash", LongType, nullable = false)
                            )),
                            shardingColumns = Array("id"),
                            pkColumns = Array("id", "geohash")
                           )
val indexSpec = IndexSpecification(
    indexName = tabName + "Index",
    tableSchema = tabSchema,
    equalColumns = Seq("id"),
    sortColumns = Seq(SortSpecification("geohash", ColumnOrder.AscendingNullsLast))
)

tabName = performance
tabSchema = TableSchema(tableName=performance, schemaName=Some(RGANTI), schema=StructType(StructField(llat,DoubleType,false), StructField(llon,DoubleType,false), StructField(ulat,DoubleType,false), StructField(ulon,DoubleType,false), StructField(id,LongType,false), StructField(geohash,LongType,false)), shardingColumns=WrappedArray(id), pkColumns=WrappedArray(id, geohash), partitionColumns=None)
indexSpec = 


IndexSpecification(indexName=performanceIndex, indexID=None,
equalColumns=(id),
sortColumns=(SortSpecification(geohash,AscendingNullsLast)),
includeColumns=())


### Drop table if table already exists

In [5]:
try {
    println("Attempt to drop table if table already exists")
    ctx.dropTable(tabName)
} catch {
    case e: Exception =>
    val TableNotFound = e.getMessage.split(" ").contains("SQLCODE=-204,")
    if (TableNotFound) {
        println("Table not found, skip dropping table...")
    } else {
        println("EXCEPTION: Exception during drop table. Trying to exit..." + e.getMessage)
        e.printStackTrace()
        sys.exit(1)
    }
}

Attempt to drop table if table already exists


None

### Create table

In [6]:
println("Creating table " + tabName)
var res = ctx.createTableWithIndex(tabSchema, indexSpec)
assert(res.isEmpty, s"create table: ${res.getOrElse("success")}")
val tab = ctx.getTable(tabName)
println("Table schema = " + tab )

Creating table performance
Table schema = ResolvedTableSchema(tableName=RGANTIPERFORMANCE, schemaName=RGANTI, tableID=28, tableGroupName=sys_RGANTIPERFORMANCE, tableGroupID=14, numShareds=36, schema=StructType(StructField(LLAT,DoubleType,false), StructField(LLON,DoubleType,false), StructField(ULAT,DoubleType,false), StructField(ULON,DoubleType,false), StructField(ID,LongType,false), StructField(GEOHASH,LongType,false)),
shardingColumns=(ID),
pkColumns=(ID,GEOHASH),
pkIndex=Some(IndexSpecification(indexName=__primaryKey, indexID=Some(0),
equalColumns=(),
sortColumns=(SortSpecification(ID,AscendingNullsLast),SortSpecification(GEOHASH,AscendingNullsLast)),
includeColumns=())),
indexes=(IndexSpecification(indexName=__primaryKey, indexID=Some(0),
equalColumns=(),
sortColumns=(SortSpecification(ID,AscendingNullsLast),SortSpecification(GEOHASH,AscendingNullsLast)),
includeColumns=()))stringLengthsMap())


res = None
tab = 


ResolvedTableSchema(tableName=RGANTIPERFORMANCE, schemaName=RGANTI, tableID=28, tableGroupName=sys_RGANTIPERFORMANCE, tableGroupID=14, numShareds=36, schema=StructType(StructField(LLAT,DoubleType,false), StructField(LLON,DoubleType,false), StructField(ULAT,DoubleType,false), StructField(ULON,DoubleType,false), StructField(ID,LongType,false), StructField(GEOHASH,LongType,false)),
shardingColumns=(ID),
pkColumns=(ID,GEOHASH),
pkIndex=Some(IndexSpecification(indexName=__primaryKey, indexID=Some(0),
equalColumns=(),
sortColumns=(SortSpecification(ID,AscendingNullsLast),SortSpecification(GEOHASH,AscendingNullsLast)),
includeColumns=())),
indexes=(IndexSpecification(indexName=__primaryKey, index...


### Insert data into table

In [7]:
spark.sparkContext.hadoopConfiguration.set("fs.s3a.access.key", "<AWS_ACCESS_KEY>")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.secret.key", "<AWS_SECRET_KEY>")

In [8]:
val df = spark.read.parquet("s3a://vz-raster-images/performance")

df = [llat: double, llon: double ... 2 more fields]


[llat: double, llon: double ... 2 more fields]

In [9]:
df.printSchema

root
 |-- llat: double (nullable = true)
 |-- llon: double (nullable = true)
 |-- ulat: double (nullable = true)
 |-- ulon: double (nullable = true)



In [10]:
df.show(3, false)

+----+------+-----+------------------+
|llat|llon  |ulat |ulon              |
+----+------+-----+------------------+
|30.0|-99.0 |30.01|-98.99            |
|30.0|-98.99|30.01|-98.97999999999999|
|30.0|-98.98|30.01|-98.97            |
+----+------+-----+------------------+
only showing top 3 rows



In [11]:
df.count

10000000

In [12]:
val geoHashUDF = udf((llat: Double, llon: Double, ulat: Double, ulon: Double) => {
    val bbox = new BoundingBoxEG(llat, llon, ulat, ulon)
    val bvs = GeoHashEG.getInstance().geoHashCoverAtDepth(bbox, bitDepth).asScala
    bvs.map(bv => (new BitVector(bv)).getLongArray()(0) >>> 1)
})

geoHashUDF = UserDefinedFunction(<function4>,ArrayType(LongType,false),Some(List(DoubleType, DoubleType, DoubleType, DoubleType)))


UserDefinedFunction(<function4>,ArrayType(LongType,false),Some(List(DoubleType, DoubleType, DoubleType, DoubleType)))

In [13]:
df.withColumn("id", monotonically_increasing_id()).withColumn("geohash", explode(geoHashUDF(col("llat"), col("llon"), col("ulat"), col("ulon")))).show(10, false)

+----+------+-----+------------------+---+-------------------+
|llat|llon  |ulat |ulon              |id |geohash            |
+----+------+-----+------------------+---+-------------------+
|30.0|-99.0 |30.01|-98.99            |0  |2838221454141554688|
|30.0|-99.0 |30.01|-98.99            |0  |2838221471321423872|
|30.0|-99.0 |30.01|-98.99            |0  |2838221488501293056|
|30.0|-99.0 |30.01|-98.99            |0  |2838221505681162240|
|30.0|-98.99|30.01|-98.97999999999999|1  |2838221471321423872|
|30.0|-98.99|30.01|-98.97999999999999|1  |2838221522861031424|
|30.0|-98.99|30.01|-98.97999999999999|1  |2838221505681162240|
|30.0|-98.99|30.01|-98.97999999999999|1  |2838221557220769792|
|30.0|-98.98|30.01|-98.97            |2  |2838221522861031424|
|30.0|-98.98|30.01|-98.97            |2  |2838221540040900608|
+----+------+-----+------------------+---+-------------------+
only showing top 10 rows



In [14]:
df.withColumn("id", monotonically_increasing_id()).withColumn("geohash", explode(geoHashUDF(col("llat"), col("llon"), col("ulat"), col("ulon"))))
.foreachPartition(partition => {
    ConfigurationReader.setDeploymentID("db2eventstore-1583771517259")
    ConfigurationReader.setEventUser("rganti")
    ConfigurationReader.setEventPassword("password")
    ConfigurationReader.setConnectionEndpoints("172.30.176.26:18729;172.30.176.26:1100")
    val ctx1 = EventContext.getEventContext("EVENTDB")
    val tab1 = ctx1.getTable(tabName)
    partition.grouped(500000).foreach(batch => {
        val future: Future[InsertResult] = ctx1.batchInsertAsync(tab1, batch.toIndexedSeq)
        val result: InsertResult = Await.result(future, Duration.Inf)
        assert(result.successful)
    })
})

### Query the table

In [15]:
import com.ibm.research.st.STConstants
import com.ibm.research.st.datamodel.geometry.ellipsoidal.{IBoundingBoxEG, IGeometryEG, IPointEG}
import com.ibm.research.st.datamodel.geometry.ellipsoidal.impl.{BoundingBoxEG, PointEG}
import com.ibm.research.st.util.{BitVector, LatLongUtil}


def getExpandedBB(geom: IGeometryEG, distance: Double): IBoundingBoxEG = {
  val bbox: IBoundingBoxEG = geom.getBoundingBox
  val centroid: IPointEG = bbox.getCenter
  val corners: java.util.List[IPointEG] = bbox.getCorners
  val latShift: Double = Math.toDegrees(distance / STConstants.DEFAULT_DATUM.r0)
  val lowerLat: Double = Math.max(STConstants.MIN_LATITUDE, corners.get(0).getLatitude - latShift)
  val upperLat: Double = Math.min(STConstants.MAX_LATITUDE, corners.get(2).getLatitude + latShift)
  val lonShift: Double = Math.toDegrees(distance / (STConstants.DEFAULT_DATUM.r0 * Math.cos(Math.toRadians(centroid.getLatitude))))
  val lowerLon: Double = LatLongUtil.addLongitude(corners.get(0).getLongitude, -lonShift)
  val upperLon: Double = LatLongUtil.addLongitude(corners.get(2).getLongitude, lonShift)
  new BoundingBoxEG(new PointEG(lowerLat, lowerLon), new PointEG(upperLat, upperLon))
}

getExpandedBB: (geom: com.ibm.research.st.datamodel.geometry.ellipsoidal.IGeometryEG, distance: Double)com.ibm.research.st.datamodel.geometry.ellipsoidal.IBoundingBoxEG


In [16]:
def getLocationLookupKey(geometry: IGeometryEG, distance: Double, depth: Int): Array[BitVector] = {
    val geohash = GeoHashEG.getInstance()
    val bvs = geohash.tightNumberHashEncode(getExpandedBB(geometry, distance))
    bvs.map(bv => {bv.truncate(depth); bv}).distinct
}

getLocationLookupKey: (geometry: com.ibm.research.st.datamodel.geometry.ellipsoidal.IGeometryEG, distance: Double, depth: Int)Array[com.ibm.research.st.util.BitVector]


In [17]:
def getUint64KeyRange(bv: BitVector, depth: Int) = {
    val start = bv.getLongArray()(0)
    val extraBitsNeeded = depth - bv.size
    val end = start + (((1L << extraBitsNeeded) - 1L) << (java.lang.Long.SIZE - depth))
    (start >>> 1, end >>> 1)
}

getUint64KeyRange: (bv: com.ibm.research.st.util.BitVector, depth: Int)(Long, Long)


In [18]:
val es = new EventSession(spark.sparkContext, "EVENTDB")
es.openDatabase()
es.setQueryReadOption("SnapshotNone")
es.loadEventTable(tabName).createOrReplaceTempView(tabName)

es = org.apache.spark.sql.ibm.event.EventSession@7c4e39e1


org.apache.spark.sql.ibm.event.EventSession@7c4e39e1

In [19]:
val condition = getLocationLookupKey(new PointEG(43, -76), 1000, bitDepth)
.map(bv => getUint64KeyRange(bv, bitDepth)).map(range => s"geohash between ${range._1} and ${range._2}").mkString(" or ")
val queryStr = s"SELECT * FROM $tabName WHERE $condition"
println(queryStr)
val results = es.sql(queryStr)
println(results.count)

SELECT * FROM performance WHERE geohash between 3668699835189428224 and 3668699886729035776 or geohash between 3668699972628381696 and 3668700024167989248 or geohash between 3668699766469951488 and 3668699818009559040 or geohash between 3668699903908904960 and 3668699955448512512
72


condition = geohash between 3668699835189428224 and 3668699886729035776 or geohash between 3668699972628381696 and 3668700024167989248 or geohash between 3668699766469951488 and 3668699818009559040 or geohash between 3668699903908904960 and 3668699955448512512
queryStr = SELECT * FROM performance WHERE geohash between 3668699835189428224 and 3668699886729035776 or geohash between 3668699972628381696 and 3668700024167989248 or geohash between 3668699766469951488 and 3668699818009559040 or geohash between 3668699903908904960 and 3668699955448512512
results = [LLAT: double, LLON: double ... 4 more fields]


[LLAT: double, LLON: double ... 4 more fields]

In [28]:
val r = scala.util.Random
val start = System.nanoTime()
1 to 1000 foreach { _ =>
    val lat = MIN_LAT + (MAX_LAT - MIN_LAT) * r.nextDouble
    val lon = MIN_LON + (MAX_LON - MIN_LON) * r.nextDouble
    val condition = getLocationLookupKey(new PointEG(lat, lon), 10000, bitDepth)
    .map(bv => getUint64KeyRange(bv, bitDepth)).map(range => s"geohash between ${range._1} and ${range._2}").mkString(" or ")
    val queryStr = s"SELECT * FROM $tabName WHERE $condition"
    es.sql(queryStr).count
}
val end = System.nanoTime()
println("Total time: " + (end - start) / 1e9 + "s")

Total time: 722.646905508s


r = scala.util.Random$@c317786d
start = 308742324034850
end = 309464970940358


309464970940358

In [26]:
val r = scala.util.Random
val start = System.nanoTime()
1 to 1000 foreach { _ =>
    val lat = MIN_LAT + (MAX_LAT - MIN_LAT) * r.nextDouble
    val lon = MIN_LON + (MAX_LON - MIN_LON) * r.nextDouble
    val condition = getLocationLookupKey(new PointEG(lat, lon), 1000, bitDepth)
    .map(bv => getUint64KeyRange(bv, bitDepth)).map(range => s"geohash between ${range._1} and ${range._2}").mkString(" or ")
    val queryStr = s"SELECT * FROM $tabName WHERE $condition"
    es.sql(queryStr).count
}
val end = System.nanoTime()
println("Total time: " + (end - start) / 1e9 + "s")

Total time: 690.243799377s


r = scala.util.Random$@c317786d
start = 307011145598258
end = 307701389397635


307701389397635

In [27]:
val r = scala.util.Random
val start = System.nanoTime()
1 to 1000 foreach { _ =>
    val lat = MIN_LAT + (MAX_LAT - MIN_LAT) * r.nextDouble
    val lon = MIN_LON + (MAX_LON - MIN_LON) * r.nextDouble
    val condition = getLocationLookupKey(new PointEG(lat, lon), 100, bitDepth)
    .map(bv => getUint64KeyRange(bv, bitDepth)).map(range => s"geohash between ${range._1} and ${range._2}").mkString(" or ")
    val queryStr = s"SELECT * FROM $tabName WHERE $condition"
    es.sql(queryStr).count
}
val end = System.nanoTime()
println("Total time: " + (end - start) / 1e9 + "s")

Total time: 497.043019402s


r = scala.util.Random$@c317786d
start = 307967706570956
end = 308464749590358


308464749590358